In [49]:
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

import numpy as np

import time
import utils
import models
import argparse
import data_loader
import pandas as pd
import ujson as json

from sklearn import metrics

from ipdb import set_trace

In [50]:
#parser = argparse.ArgumentParser()
#parser.add_argument('--epochs', type=int, default=1000)
#parser.add_argument('--batch_size', type=int, default=32)
#arser.add_argument('--model', type=str)
#parser.add_argument('--hid_size', type=int)
#parser.add_argument('--impute_weight', type=float)
#parser.add_argument('--label_weight', type=float)
#args = parser.parse_args()

from collections import namedtuple
args = namedtuple("MyStruct", "model epochs batch_size hid_size impute_weight label_weight")

args.model="rits_i"
args.epochs=1000
args.batch_size=32
args.hid_size=108
args.impute_weight=0.3
args.label_weight=1.0

In [51]:
def train(model):
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    print('\r Loading data...')
    data_iter = data_loader.get_loader(batch_size=args.batch_size)

    for epoch in range(args.epochs):
        model.train()

        run_loss = 0.0

        for idx, data in enumerate(data_iter):
            data = utils.to_var(data)
            ret = model.run_on_batch(data, optimizer, epoch)

            run_loss += ret['loss'].item()

            print('\r Progress epoch {}, {:.2f}%, average loss {}'.format(epoch, (idx + 1) * 100.0 / len(data_iter), run_loss / (idx + 1.0)))

        evaluate(model, data_iter)

In [52]:
def evaluate(model, val_iter):
    model.eval()

    labels = []
    preds = []

    evals = []
    imputations = []

    save_impute = []
    save_label = []

    for idx, data in enumerate(val_iter):
        data = utils.to_var(data)
        ret = model.run_on_batch(data, None)

        # save the imputation results which is used to test the improvement of traditional methods with imputed values
        save_impute.append(ret['imputations'].data.cpu().numpy())
        save_label.append(ret['labels'].data.cpu().numpy())

        pred = ret['predictions'].data.cpu().numpy()
        label = ret['labels'].data.cpu().numpy()
        is_train = ret['is_train'].data.cpu().numpy()

        eval_masks = ret['eval_masks'].data.cpu().numpy()
        eval_ = ret['evals'].data.cpu().numpy()
        imputation = ret['imputations'].data.cpu().numpy()

        evals += eval_[np.where(eval_masks == 1)].tolist()
        imputations += imputation[np.where(eval_masks == 1)].tolist()

        # collect test label & prediction
        pred = pred[np.where(is_train == 0)]
        label = label[np.where(is_train == 0)]

        labels += label.tolist()
        preds += pred.tolist()

    labels = np.asarray(labels).astype('int32')
    preds = np.asarray(preds)

    print('AUC {}'.format(metrics.roc_auc_score(labels, preds)))

    evals = np.asarray(evals)
    imputations = np.asarray(imputations)

    print('MAE', np.abs(evals - imputations).mean())

    print('MRE', np.abs(evals - imputations).sum() / np.abs(evals).sum())

    save_impute = np.concatenate(save_impute, axis=0)
    save_label = np.concatenate(save_label, axis=0)

    np.save('./result/{}_data'.format(args.model), save_impute)
    np.save('./result/{}_label'.format(args.model), save_label)

In [53]:
def run():
    model = getattr(models, args.model).Model(args.hid_size, args.impute_weight, args.label_weight)
    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print('Total params is {}'.format(total_params))

    if torch.cuda.is_available():
        model = model.cuda()

    train(model)

In [54]:
if __name__ == '__main__':
    run()

Total params is 85572
 Loading data...


TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "D:\Anaconda3\lib\site-packages\torch\utils\data\_utils\worker.py", line 202, in _worker_loop
    data = fetcher.fetch(index)
  File "D:\Anaconda3\lib\site-packages\torch\utils\data\_utils\fetch.py", line 47, in fetch
    return self.collate_fn(data)
  File "D:\GitHub\BRITS\data_loader.py", line 57, in collate_fn
    ret_dict = {'forward': to_tensor_dict(forward), 'backward': to_tensor_dict(backward)}
  File "D:\GitHub\BRITS\data_loader.py", line 47, in to_tensor_dict
    masks = torch.FloatTensor(map(lambda r: r['masks'], recs))
TypeError: new(): data must be a sequence (got map)


In [56]:
model = getattr(models, args.model).Model(args.hid_size, args.impute_weight, args.label_weight)
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

In [64]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)

print('\r Loading data...')
data_iter = data_loader.get_loader(batch_size=args.batch_size)

 Loading data...


In [76]:
rec=data_iter.dataset.__getitem__(1)
recs=data_iter.dataset
mask = torch.FloatTensor(list(map(lambda r: r['masks'], rec)))

TypeError: string indices must be integers

In [78]:
rec['masks']

KeyError: 'masks'

In [59]:
model.train()

Model(
  (rnn_cell): LSTMCell(70, 108)
  (regression): Linear(in_features=108, out_features=35, bias=True)
  (temp_decay): TemporalDecay()
  (out): Linear(in_features=108, out_features=1, bias=True)
)

In [61]:
data=1

for idx, data1 in enumerate(data_iter):
    data = utils.to_var(data1)
    ret = model.run_on_batch(data, optimizer, 1)

    run_loss += ret['loss'].item()



IndexError: too many indices for tensor of dimension 1

In [62]:
data1.items()

dict_items([('forward', {'values': tensor([[[ 0.0000,  1.9268,  0.0000,  ..., -0.4321,  0.0000,  0.0000],
         [ 0.0000,  1.5333,  0.0000,  ..., -0.5896,  0.0000,  0.0000],
         [ 0.0000,  1.4208,  0.0000,  ..., -0.0282,  0.0000,  0.0000],
         ...,
         [ 0.0000,  0.8025,  0.0000,  ..., -0.1406,  0.0000,  0.0000],
         [ 0.0000,  0.8025,  0.0000,  ..., -0.1628,  0.0000,  0.0000],
         [ 0.0000,  1.0274,  0.0000,  ...,  0.0835,  0.0000,  0.0000]],

        [[ 0.0000,  0.0000, -1.3633,  ...,  0.0000, -0.1254, -0.4536],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 0.0000,  0.4090,  0.0000,  ..., -0.2530,  0.0000,  0.0000],
         [ 0.0000,  1.1398,  0.0000,  ...,  0.1959,  0.0000,  0.0000],
         [ 0.0000,  0.2404,  0.0000,  ..., -0.2753,  0.0000,  0.0000]],

        [[ 0.0000,  0.2966, -0.0134,  ...,  1.4749, -0.3697, -0.2372],
         [

In [63]:
values=data1['forward']['masks']
values




tensor([])